# Scraping data

* ## Import relevant packages

In [61]:
#%reset

In [62]:

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from tqdm import tqdm
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import json
import pandas as pd
import csv
import time
import requests
import concurrent.futures
import lxml
from datetime import datetime
from datetime import timedelta
from ast import literal_eval
from selenium.webdriver.common.keys import Keys
import matplotlib.pyplot as plt
import math

pd.set_option('display.max_columns', None) #We want to see all columns of our dataset. 



In [86]:
#We make a function that read our saved excel files. And make it a df and remove the row index from when we saved the file. 

def read_saved_excel(Excel_string_name):
    read_df = pd.read_excel(Excel_string_name)    #Get the data from our saved excel
    read_df = pd.DataFrame(read_df) #make it to a dataframe in Python, with the Pandas inbuild function
    read_df.drop('Unnamed: 0', axis=1, inplace=True) #Remove the created row index from when we saved the file
    return read_df

* ## Look at data

In [330]:
data= pd.read_excel("Data u besk_endelig.xlsx")
data.columns



Index(['case_id', 'type', 'energy_law_applies', 'energy_label_classification',
       'presale', 'num_floors', 'num_rooms', 'living_area', 'basement_area',
       'lot_area', 'year_built', 'sold_price', 'registered_price',
       'address_id', 'rental', 'latest_status', 'weighted_area',
       'closed_or_sold_at', 'original_price', 'unit_id', 'usage_code',
       'living_area.1', 'business_area', 'num_rooms.1', 'updated_at',
       'address_id.1', 'num_floors.1', 'year_built.1', 'year_renovated',
       'ownership', 'building_id', 'num_toilets', 'toilet_condition_code',
       'num_bathrooms', 'bathroom_condition_code', 'kitchen_condition_code',
       'floor_id', 'status', 'property_number', 'building_id.1',
       'building_number', 'entry_address_id', 'usage_code.1', 'year_built.2',
       'year_renovated.1', 'outer_wall_code', 'roof_code',
       'additional_outer_wall_mat', 'additional_roof_mat',
       'asbestos_materials', 'total_area', 'living_area_total',
       'business_area

'postnradgadr', 'postnummernavn', 'kommune', 'vejnavn', 'husnr', 'dør', 'etage'



test3 = dingeo_page('https://www.dingeo.dk/adresse/2450-København-SV/rektorparken-14/1-tv')

In [331]:
df_data = pd.DataFrame(data)



In [442]:
def add_ID_count(df_input):
    df = df_input
    for i in range(0, len(df)):
        df.loc[i, "ID_Count"] = i

    return df

In [333]:
df_data = add_ID_count(df_data)

* ## Code for MinGeo.dk

Cleaning the data and get links to MinGeo.dk :

In [443]:
def get_links_to_dingeo(df_input):
    
    df = df_input
    df["dingeo_link"] = ""

    for i in range(0, len(df)):

        
        
        vejnavn = df['vejnavn'][i]
        postnummernavn = df['postnummernavn'][i]
        etage = str(df['etage'][i])

        if '-' in vejnavn: #If '-' we need to make it a double '--'
            vejnavn = vejnavn.replace('-', '--')

        if ' ' in vejnavn: #Remove space and replace with '-'
            vejnavn = vejnavn.replace(' ', '-')
        
        if ' ' in postnummernavn: #Remove space and replace with '-'
            postnummernavn = postnummernavn.replace(' ', '-')

        if '.0' in etage: #If '.0', then replace with nothing
            etage = etage.replace('.0', '')

        if '10' in etage: #then nothing
            pass
        elif '20' in etage: #then nothing
            pass
        elif '30' in etage: #then nothing
            pass
        elif '0' in etage: 
            etage = etage.replace('0', 'st')
        else:
            pass

        if '-1' in etage: #replace '-1' with kl, as in 'kælder'
            etage = etage.replace('-1', 'kl')
        
        

        if 'nan' in str(df['etage'][i]): #Then we do not need etage and dør in our link
            url = 'https://www.dingeo.dk/adresse/' + str(df['postnradgadr'][i]) + '-' + str(postnummernavn) + '/' + str(vejnavn) + '-' + str(df['husnr'][i])  
        elif 'nan' in str(df['dør'][i]):
            url = 'https://www.dingeo.dk/adresse/' + str(df['postnradgadr'][i]) + '-' + str(postnummernavn) + '/' + str(vejnavn) + '-' + str(df['husnr'][i]) + '/' + str(etage) 
        else:
            url = 'https://www.dingeo.dk/adresse/' + str(df['postnradgadr'][i]) + '-' + str(postnummernavn) + '/' + str(vejnavn) + '-' + str(df['husnr'][i]) + '/' + str(etage) + '-' + str(df['dør'][i])

        df['dingeo_link'][i] = url

    return df

We define our webpage scrap function

In [335]:
def dingeo_page(df_input, count):

    url = df_input.loc[count, 'dingeo_link']

    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, 'html.parser')

    # Dictionary
    data = {}
    data['dingeo_link'] = url

    try:
        if 'Fejl På Adressen' in str(soup.find_all("div", {"class": 'intro'})[0].find_all("h1")[0].get_text()): 
            data['address_id'] = df_input.loc[count, 'address_id']
            data['Radonrisiko'] = ['Missing']
            data['Støjmåling'] = ['Missing']
            data['Oversvømmelsesrisiko_skybrud'] = ['Missing']
            data['Meter_over_havet'] =['Missing']
            data['Indbrudsrisiko'] =['Missing']
            data['Bevaringsværdi'] = ['Missing']
            data['Største_parti'] =['Missing']
            data['Valgdeltagelse'] = ['Missing']
            data['Afstemningsområde']= ['Missing']
            data['AVM_pris'] = ['Missing']
        
        else:
            data['address_id'] = df_input.loc[count, 'address_id']
            try:
                data['Radonrisiko'] = [soup.find_all("div", {"id": 'radon'})[0].find_all("strong")[0].get_text()]
            except:
                data['Radonrisiko'] = ['Missing']

            try:
                if 'ikke registreret trafikstøj' in soup.find_all("div", {"id": 'trafikstoej'})[0].get_text():
                    data['Støjmåling'] = ['Ingen trafikstøj']
                elif 'mangler desværre at indsamle trafikstøj' in soup.find_all("div", {"id": 'trafikstoej'})[0].get_text():
                    data['Støjmåling'] = ['Missing']
                else:
                    data['Støjmåling'] = [soup.find_all("div", {"id": 'trafikstoej'})[0].find_all("b")[1].get_text()]
            except:
                data['Støjmåling'] = ['Missing']
            
            try:
                data['Oversvømmelsesrisiko_skybrud'] = [soup.find_all("div", {"id": 'skybrud'})[0].find_all("b")[0].get_text()]
            except:
                data['Oversvømmelsesrisiko_skybrud'] = ['Missing']
            
            try:
                data['Meter_over_havet'] = [soup.find_all("div", {"id": 'stormflod'})[0].find_all("b")[0].get_text()]
            except:
                data['Meter_over_havet'] = ['Missing']
            


            
            
            try:

                data['Indbrudsrisiko'] = [soup.find_all("div", {"id": 'lokalomraadet'})[0].find_all("u")[1].get_text()]
            except:
                data['Indbrudsrisiko'] = ['Missing']

            try:
                if 'img loading="lazy" src="/img/fbb/icon-1.gif"' in str(soup.find_all('img', src='/img/fbb/icon-1.gif')):
                    data['Bevaringsværdi'] = [1]
                elif 'img loading="lazy" src="/img/fbb/icon-2.gif"' in str(soup.find_all('img', src='/img/fbb/icon-2.gif')):
                    data['Bevaringsværdi'] = [2]
                elif 'img loading="lazy" src="/img/fbb/icon-3.gif"' in str(soup.find_all('img', src='/img/fbb/icon-3.gif')):
                    data['Bevaringsværdi'] = [3]  
                elif 'img loading="lazy" src="/img/fbb/icon-4.gif"' in str(soup.find_all('img', src='/img/fbb/icon-4.gif')):
                    data['Bevaringsværdi'] = [4]  
                elif 'img loading="lazy" src="/img/fbb/icon-5.gif"' in str(soup.find_all('img', src='/img/fbb/icon-5.gif')):
                    data['Bevaringsværdi'] = [5]
                elif 'img loading="lazy" src="/img/fbb/icon-6.gif"' in str(soup.find_all('img', src='/img/fbb/icon-6.gif')):
                    data['Bevaringsværdi'] = [6]
                elif 'img loading="lazy" src="/img/fbb/icon-7.gif"' in str(soup.find_all('img', src='/img/fbb/icon-7.gif')):
                    data['Bevaringsværdi'] = [7]
                elif 'img loading="lazy" src="/img/fbb/icon-8.gif"' in str(soup.find_all('img', src='/img/fbb/icon-8.gif')):
                    data['Bevaringsværdi'] = [8]
                elif 'img loading="lazy" src="/img/fbb/icon-9.gif"' in str(soup.find_all('img', src='/img/fbb/icon-9.gif')):
                    data['Bevaringsværdi'] = [9]
                elif 'img loading="lazy" src="/img/fbb/icon-x.gif"' in str(soup.find_all('img', src='/img/fbb/icon-x.gif')): 
                    data['Bevaringsværdi'] = [0]
                else:
                    data['Bevaringsværdi'] = 'Unknown'
            except:
                data['Bevaringsværdi'] = ['Missing']


            try:
                data['Største_parti'] = re.findall(r'valg/(.*?)(?<!\\).png',
                                                str(soup.find_all("div", {"id": 'valgdata'})[0].find_all('h2')[0]))
                data['Valgdeltagelse'] = \
                re.findall("\d+.\d+", str(soup.find_all("div", {"id": 'valgdata'})[0].find_all('p')[1]))[1]
                data['Afstemningsområde'] = [soup.find_all("div", {"id": 'valgdata'})[0].find_all("strong")[0].get_text()]
            except:
                data['Største_parti'] = ['Missing']
                data['Valgdeltagelse'] = ['Missing']
                data['Afstemningsområde'] = ['Missing']

            try:
                url_vurdering = url + '/vurdering'
                resp_vurdering = requests.get(url_vurdering)
                soup_vurdering = BeautifulSoup(resp_vurdering.text, 'html.parser')
                data['AVM_pris'] = soup_vurdering.find_all("div", {"id": 'avmnumber'})[0].get_text()  # made correction
            except:
                data['AVM_pris'] = ['Missing']
    except:
            data['address_id'] = df_input.loc[count, 'address_id']
            data['Radonrisiko'] = ['Missing']
            data['Støjmåling'] = ['Missing']
            data['Oversvømmelsesrisiko_skybrud'] = ['Missing']
            data['Meter_over_havet'] =['Missing']
            data['Indbrudsrisiko'] =['Missing']
            data['Bevaringsværdi'] = ['Missing']
            data['Største_parti'] =['Missing']
            data['Valgdeltagelse'] = ['Missing']
            data['Afstemningsområde']= ['Missing']
            data['AVM_pris'] = ['Missing']

        # Make dataframe
    df_page = pd.DataFrame(data)

    return df_page

In [444]:

def dingeo_merged(df_input):

     d1 = df_input
     d1 = d1.drop_duplicates(subset=['address_id']) #remove duplicates
     d1 = d1.reset_index() #We reset the index number, so we can use our functions

     d1_url = get_links_to_dingeo(d1) #We get the links
     d1_url = d1_url.loc[: ,['address_id', 'dingeo_link']]
     
     for i in tqdm(range(0, len(d1_url))):
                 
          a = dingeo_page(d1_url, i)
                    
          if i == 0 :
               new_df =  a

          else :
               new_df = pd.concat([new_df, a])
               
     
     merged = pd.merge(df_input, new_df, how='inner', on = 'address_id')

     return merged

* ## Code for HvorLangtErDer.dk

In [445]:
#### Scrape information for single address from hvorlangterder.dk

def get_hvorlangterder(address):
    try:
        url = 'https://hvorlangterder.poi.viamap.net/v1/nearestpoi/?poitypes=daycare,doctor,hospital,junction,metro,school,stop,strain,supermarket,train,library,pharmacy,coast,forest,lake,airport,sportshall,publicbath,soccerfield,roadtrain&fromaddress=' + address + '&mot=foot&token=eyJkcGZ4IjogImh2b3JsYW5ndGVyZGVyIiwgInByaXZzIjogInIxWjByMEYwazZCdFdxUWNPVXlrQi95NlNVcEp2MlFiZ3lYZXRxNEhZNFhPLzNZclcwK0s5dz09In0.fP4JWis69HmaSg5jVHiK8nemiCu6VaMULSGGJyK4D4PkWq4iA1+nSHWMaHxepKwJ83sEiy9nMNZhv7BcktRNrA'
        resp = requests.get(url)
        cont = resp.json()
        df = pd.DataFrame(cont).loc[['routedmeters']]
        df['Location'] = address
        
        return (df)
    except Exception:
            df = pd.DataFrame(columns=['school', 'roadtrain', 'junction', 'daycare', 'metro', 'doctor', 'soccerfield', 'hospital', 'stop', 'lake', 'supermarket', 'pharmacy', 'strain', 'airport', 'train', 'library', 'publicbath', 'coast', 'sportshall', 'forest', 'Location'], index=['routedmeters'])
            return (df)
            



   


In [446]:
	 #### Scrape data from hvorlangterder.dk for all adresses and merge with data from Boligsiden and DinGeo.dk
def add_hvorlangterder(df_input):

    new_df = pd.DataFrame()
    for i in tqdm(range(0, len(df_input))):
        
        address = str(df_input.loc[i, 'vejnavn']) + str(df_input.loc[i, 'husnr'])  + str(df_input.loc[i, 'kommune'])
        
        
        new_df = get_hvorlangterder(str(address))

           
        new_df["ID_Count"] = i
            
        if i == 0 :
            df_merged =  new_df
            
        else :
            df_merged = pd.concat([df_merged, new_df])
           
    merged = pd.merge(df_input, df_merged, how='inner', on = 'ID_Count')
    return merged


* ## Lets get our scrapped data!

* ### Scraping HvorLangtErDer.dk

In [339]:
# This line took about 7 to 8 hours to run. 

#hvorlangterder_data_1 = add_hvorlangterder(df_data) 

In [340]:
#We save the new created file, with the data from hvorlangterder. 
#So we dont need to run it again. It is a heavy task. 

#hvorlangterder_data_1.to_excel("hvorlangterder_data_1.xlsx") 

In [341]:
#we can here read our saved excel file
hvorlangterder_data_1 =read_saved_excel("hvorlangterder_data_1.xlsx") 

We split our dataset, so if there are any mistakes, we can learn it "faster".

In [342]:
#We now split the data in 6 dataset, to minmize the risk of failure. The dingeo could fail, if it lose internet connection.. Yes, it did happen.. 

d=hvorlangterder_data_1
d1 = d.loc[0:7500,]
d2 = d.loc[7501:17009,]
d3 = d.loc[17010:25509,]
d4 = d.loc[25510:34009,]
d5 = d.loc[34010:43226,]
d6 = d.loc[43227:52443,]

In [343]:
#dingeo_d1 = dingeo_merged(d1)
# 
#  #6056/6056 [6:55:56<00:00,  4.12s/it] - Remember it removes duplicates in the for-loop.

In [344]:
#dingeo_d1.to_excel("dingeo_d1.xlsx") 

In [345]:
#dingeo_d2 = dingeo_merged(d2) 
# 
# #7721/7721 [15:35:26<00:00,  7.27s/it] - Remember it removes duplicates in the for-loop.

In [346]:
#dingeo_d2.to_excel("dingeo_d2.xlsx") 

In [347]:
#dingeo_d3 = dingeo_merged(d3) 
# 
# #6896/6896 [8:51:08<00:00,  4.62s/it] - Remember it removes duplicates in the for-loop.

In [348]:
#dingeo_d3.to_excel("dingeo_d3.xlsx")

In [349]:
#dingeo_d4 = dingeo_merged(d4) 
# 
# #6935/6935 [13:01:51<00:00,  6.76s/it] - Remember it removes duplicates in the for-loop.

In [350]:
#dingeo_d4.to_excel("dingeo_d4.xlsx")

In [351]:
#dingeo_d5 = dingeo_merged(d5) 
# 
# #7514/7514 [13:48:39<00:00,  6.62s/it] 

In [352]:
#dingeo_d5.to_excel("dingeo_d5.xlsx")

In [353]:
#dingeo_d6 = dingeo_merged(d6) 
# 
# #7430/7430 [13:02:45<00:00,  6.32s/it] 

In [354]:
#dingeo_d6.to_excel("dingeo_d6.xlsx")

* #### Get Missing rows

We now need to pd.concat the files, so we get one big raw dataset. 

Also we are aware of some rows are missing. I think that this has happened due connection error to the ethernet. But I have located these rows manually, and will also connect these. 

It happened because of the pass-function. But I found the problem. 

In [355]:
List_index = pd.DataFrame([9026, 9901, 9970, 11465, 15456, 15457, 15862, 15863, 15864, 26610, 31165, 31167, 32749, 32750], columns = ['Missing Index'])
List_index

,Missing Index
0,9026
1,9901
2,9970
3,11465
4,15456
5,15457
6,15862
7,15863
8,15864
9,26610


In [447]:
#We make a function, where we give the hvorlangterder_data_1 as input and the index_list from above. 

#And then we get our missing rows.

def get_missing_rows(df_input, Index_list):
    
    df = df_input
    url_df = get_links_to_dingeo(df)
    
    for i in tqdm(range(0, len(Index_list))):
        a = pd.DataFrame([list(df.loc[Index_list.loc[i,'Missing Index']])], columns= list(df.columns) )
        b = dingeo_page(url_df, Index_list.loc[i,'Missing Index'])
        c = pd.merge(a, b, how = 'inner', on = 'address_id')
        
        if i == 0:
            new_df = c
        else:
            new_df = pd.concat([new_df, c])
    return new_df

In [357]:
#we get our 14 missing rows

missing_rows = get_missing_rows(hvorlangterder_data_1, List_index) # 14/14 [01:24<00:00,  6.01s/it]

C:\Users\Mtubo\AppData\Local\Temp\ipykernel_17752\3216548660.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dingeo_link'][i] = url
100%|██████████| 14/14 [01:21<00:00,  5.81s/it]


* ## Merge all data togehter

We will now merge(concat) all the data together as one file, like the one we where given. So we will order the data by our column "ID_Count".
So we have all our raw data as we got it, and with the extra variables. 

In [358]:
dingeo_d1 = read_saved_excel('dingeo_d1.xlsx')
dingeo_d2 = read_saved_excel('dingeo_d2.xlsx')
dingeo_d3 = read_saved_excel('dingeo_d3.xlsx')
dingeo_d4 = read_saved_excel('dingeo_d4.xlsx')
dingeo_d5 = read_saved_excel('dingeo_d5.xlsx')
dingeo_d6 = read_saved_excel('dingeo_d6.xlsx')


In [359]:
#raw_data1 = pd.concat([dingeo_d1, dingeo_d2, dingeo_d3, dingeo_d4, dingeo_d5, dingeo_d6, missing_rows], axis=0) # concat along rows


In [360]:
#We sort our dataset 

#raw_data1.sort_values(by= ['ID_Count'], inplace=True) 

#We reset the index number, as we started with in the begining.

#raw_data1 = raw_data1.reset_index() 

In [361]:
#We remove the index row, that was created when we reset the index, and also the dingeo_link_x and *y, which has no values

#raw_data1.drop('index', axis=1, inplace=True)
#raw_data1.drop('dingeo_link_x', axis=1, inplace=True)
#raw_data1.drop('dingeo_link_y', axis=1, inplace=True)


* ## Add extra data from Boligsiden.dk

In [15]:
extra_data = pd.read_excel("vurderinger.xlsx")


In [21]:
extra_data.drop(columns=['living_area','address_type','year_built','year_renovated', 'num_floors', 'num_rooms'], inplace=True)

In [28]:
raw_data11 = pd.merge(raw_data1, extra_data, how = 'left', on = ['address_id', 'gstkvhx_kommunekode_vejkode_husnr_x'])

In [31]:
raw_data11['latest_prop_valuation'].isna().sum()

5878

In [32]:
#And we save our raw dataframe as an excel.


#raw_data11.to_excel("raw_data1.xlsx")

* ## Add data from StatBank.dk

In [33]:
pd.set_option('display.max_columns',None)
#pd.reset_option('^display.', silent=True)

In [34]:
raw_data1 = read_saved_excel("raw_data1.xlsx")

We convert the column to pandas datetime and then make a new column with Year-Week

In [35]:
def dates(df_input):
    df = df_input

    year = pd.to_datetime(df['closed_or_sold_at']).dt.isocalendar().year
    month = pd.to_datetime(df['closed_or_sold_at']).dt.month
    week = pd.to_datetime(df['closed_or_sold_at']).dt.isocalendar().week
    quarter = pd.to_datetime(df['closed_or_sold_at']).dt.quarter
    df['Year_Week']= np.nan
    df['Year_Months']= np.nan
    df['Year_Quarter']= np.nan
    for i in tqdm(range(0, len(df))):
        
        if week[i] == 52 and month[i] == 1:            
            df['Year_Week'][i] = str(year[i] + 1)  + 'W01'  #We make Year-Week date for closed or sold date
            df['Year_Months'][i] = str(year[i] + 1) + 'M0' + str(month[i]) #We make Year-Week date for closed or sold date
            df['Year_Quarter'][i] = str(year[i] + 1) + 'Q' + str(1)
        elif week[i] <= 9:
            if month[i] <=9:
                df['Year_Week'][i] = str(year[i] )  + 'W0' + str(week[i])  #We make Year-Week date for closed or sold date
                df['Year_Months'][i] = str(year[i] ) + 'M0' + str(month[i]) #We make Year-Week date for closed or sold date
                df['Year_Quarter'][i] = str(year[i]) + 'Q' + str(quarter[i])
            else:
                df['Year_Week'][i] = str(year[i] )  + 'W0' + str(week[i])  #We make Year-Week date for closed or sold date
                df['Year_Months'][i] = str(year[i] ) + 'M' + str(month[i]) #We make Year-Week date for closed or sold date
                df['Year_Quarter'][i] = str(year[i]) + 'Q' + str(quarter[i])
        elif month[i] <=9:
                df['Year_Week'][i] = str(year[i] )  + 'W' + str(week[i])  #We make Year-Week date for closed or sold date
                df['Year_Months'][i] = str(year[i] ) + 'M0' + str(month[i]) #We make Year-Week date for closed or sold date
                df['Year_Quarter'][i] = str(year[i] ) + 'Q' + str(quarter[i])
        else:
            df['Year_Week'][i] = str(year[i]) + 'W' + str(week[i]) #We make Year-Week date for closed or sold date
            df['Year_Months'][i] = str(year[i]) + 'M' + str(month[i]) #We make Year-Week date for closed or sold date
            df['Year_Quarter'][i] = str(year[i]) + 'Q' + str(quarter[i])

    return df

In [36]:
raw_data = dates(raw_data1)

  0%|          | 0/52444 [00:00<?, ?it/s]C:\Users\Mtubo\AppData\Local\Temp\ipykernel_24612\3760785205.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year_Week'][i] = str(year[i] )  + 'W' + str(week[i])  #We make Year-Week date for closed or sold date
C:\Users\Mtubo\AppData\Local\Temp\ipykernel_24612\3760785205.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Year_Months'][i] = str(year[i] ) + 'M0' + str(month[i]) #We make Year-Week date for closed or sold date
C:\Users\Mtubo\AppData\Local\Temp\ipykernel_24612\3760785205.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [88]:
Obligationsrente = pd.read_excel("obligationsrente_fida_uge36-2022_REDIGERET.xlsx")
unemployed  = pd.read_excel("AUS08_redigeret.xlsx")
Population = pd.read_excel("FOLK1A_redigeret.xlsx")
OMXC25 = pd.read_excel("MPK13_redigeret.xlsx")
ConsumerPriceIndex = pd.read_excel("PRIS111_rediger.xlsx")
ForcedSales = pd.read_excel("TVANG1_redigeret.xlsx")
DNRNURI = pd.read_excel("DNRNURI_redigeret.xlsx")
ElPrice = pd.read_excel("elprisdata_oestDanmark_redigeret.xlsx")
BenzPrice = pd.read_excel("BenzinPrice_redigeret.xlsx")
DieselPrice = pd.read_excel("DieselPrice_redigeret.xlsx")
coordinates = pd.read_excel("koordinater_mathias.xlsx")

In [38]:
def insertdata(df_input, df_Obligationsrente, df_unemployed, df_population, df_OMXC25, df_ConsumerPriceIndex, df_ForcedSales, df_DNRNURI, df_ElPrice, df_BenzPrice, df_DieselPrice):
    
    a_df = pd.merge(df_input, df_Obligationsrente, how = 'left', on = 'Year_Week')

    b_df =  pd.merge(a_df, df_OMXC25, how = 'left', on = 'Year_Months')

    c_df =  pd.merge(b_df, df_ConsumerPriceIndex, how = 'left', on = 'Year_Months')

    d_df =  pd.merge(c_df, df_ForcedSales, how = 'left', on = 'Year_Months')

    e_df =  pd.merge(d_df, df_DNRNURI, how = 'left', on = 'Year_Months')

    f_df =  pd.merge(e_df, df_ElPrice, how = 'left', on = 'Year_Months')

    g_df =  pd.merge(f_df, df_BenzPrice, how = 'left', on = 'Year_Months')

    h_df =  pd.merge(g_df, df_DieselPrice, how = 'left', on = 'Year_Months')

    i_df = pd.merge(h_df, df_unemployed, how = 'left', on = 'Year_Months')

    df = pd.merge(i_df, df_population, how = 'left', on = 'Year_Quarter')
    
    

    df['unemployedRate']= np.nan
    df['Population']= np.nan
    for i in tqdm(range(0, len(df))):
        if df.loc[i, 'kommune'] == 'Frederiksberg':
            df.loc[i, 'unemployedRate'] = df.loc[i, 'unemploymentRate_Frederiksberg']
            df.loc[i, 'Population'] = df.loc[i, 'Frederiksberg_Population']
        else:
            df.loc[i, 'unemployedRate'] = df.loc[i, 'unemploymentRate_Copenhagen']
            df.loc[i, 'Population'] = df.loc[i, 'Copenhagen_Population']
    
    df.sort_values(by= ['ID_Count'], inplace=True) 

    

    #We frop Frederiksberg and copenhagen columns, so we only have the rates and number for the given "kommune" the adresse is in.
    
    df.drop(columns=['unemploymentRate_Frederiksberg', 'Frederiksberg_Population', 'unemploymentRate_Copenhagen', 'Copenhagen_Population'], inplace=True) 

    return df
    

In [39]:
raw_data2 = insertdata(raw_data, Obligationsrente, unemployed, Population, OMXC25, ConsumerPriceIndex, ForcedSales , DNRNURI, ElPrice, BenzPrice, DieselPrice)

100%|██████████| 52444/52444 [00:07<00:00, 6633.25it/s]


In [40]:
#We save the final raw data
#raw_data2.to_excel("raw_data2.xlsx")

In [41]:
#raw_data2 = read_saved_excel("raw_data2.xlsx")

* ## We just remove 'Missing', '-' and 'Mangler' from DinGeo data. 

In [42]:
raw_data2.replace('Missing', np.nan, inplace=True)

In [43]:
raw_data2.replace('Mangler', np.nan, inplace=True)

In [44]:
raw_data2['AVM_pris'].replace(' - ', np.nan, inplace=True)

* ## Balcony and Terrace ( Altan og Terrasse )

In [45]:
Beskrivelser = pd.read_excel("Beskrivelser.xlsx")

In [46]:
Beskrivelser

#We can here see that we have 52445 rows, which is one more that the other dataset we were given. 

,case_id,title,description
0,da3c0f7d-e863-4e30-95f7-201a96065ec7,NaN,\n\n\n\nLeder du efter god plads og højloftede...
1,9c5e7ed9-95f9-41c0-900f-b5e0d320e04d,NaN,NaN
2,d7e43f7e-edb6-46a6-a766-de1ddfa0e284,NaN,DEJLIG VILLALEJLIGHED MED PRIVAT LILLE HAVESTY...
3,90be8918-b3ba-40d1-8187-15f997c263e0,IDEEL ANPART - KAN IKKE BELÅNES! ENESTÅENDE H...,Området omkring Frederiksberg Allé er spækket ...
4,c2faf077-406a-4f1b-acd0-719c9e9946ab,4-VÆRELSES ALTANLEJLIGHED MIDT PÅ FREDERIKSBERG,"Stuklofter, sydvendt altan, fire skønne værels..."
...,...,...,...
52440,59ca6ff6-d547-4195-967b-3d6b1db8a38c,NaN,ALTAN-LEJLIGHED - som mezzanin-lejlighed (i 1....
52441,5f110503-f423-4510-9977-3ec14788495a,NaN,Attraktivt beliggende i det populære 'Jydekvar...
52442,59ca6ff6-d547-4195-967b-3d6b1db8a38c,NaN,ALTAN-LEJLIGHED - som mezzanin-lejlighed (i 1....
52443,5f110503-f423-4510-9977-3ec14788495a,NaN,Attraktivt beliggende i det populære 'Jydekvar...


We need to get the adress_id variable, so we get this:

In [47]:
Beskrivelser = pd.merge(Beskrivelser, raw_data2[['case_id','address_id']].drop_duplicates(),  how= 'left', on='case_id' )

In [48]:
#We look for NaN in adress_id and we can see there is 1. (This must be the extra row we were given)
Beskrivelser.isna().sum()

case_id            0
title          25070
description     1365
address_id         1
dtype: int64

In [49]:
#We look at the row
Beskrivelser[Beskrivelser['address_id'].isna() == True ]

,case_id,title,description,address_id
43712,bff6c83b-9571-4c31-87d1-fa1b19198aea,Stilren lejlighed med altan tæt på metro,Velkommen til denne fantastisk velbeliggende 2...,NaN


In [50]:
#And we remove it from our dataset: Beskrivelser
Beskrivelser = Beskrivelser.drop(43712)

In [51]:
#And we can see it is gone. 
Beskrivelser[Beskrivelser['address_id'].isna() == True ]

,case_id,title,description,address_id


In [52]:
Beskrivelser = Beskrivelser.reset_index()
Beskrivelser = Beskrivelser.drop(columns=['index'])

In [53]:
Beskrivelser

,case_id,title,description,address_id
0,da3c0f7d-e863-4e30-95f7-201a96065ec7,NaN,\n\n\n\nLeder du efter god plads og højloftede...,0a3f509d-26a5-32b8-e044-0003ba298018
1,9c5e7ed9-95f9-41c0-900f-b5e0d320e04d,NaN,NaN,0a3f509d-2680-32b8-e044-0003ba298018
2,d7e43f7e-edb6-46a6-a766-de1ddfa0e284,NaN,DEJLIG VILLALEJLIGHED MED PRIVAT LILLE HAVESTY...,0a3f50a2-a0c3-32b8-e044-0003ba298018
3,90be8918-b3ba-40d1-8187-15f997c263e0,IDEEL ANPART - KAN IKKE BELÅNES! ENESTÅENDE H...,Området omkring Frederiksberg Allé er spækket ...,0a3f50a2-a15d-32b8-e044-0003ba298018
4,c2faf077-406a-4f1b-acd0-719c9e9946ab,4-VÆRELSES ALTANLEJLIGHED MIDT PÅ FREDERIKSBERG,"Stuklofter, sydvendt altan, fire skønne værels...",0a3f50a2-a160-32b8-e044-0003ba298018
...,...,...,...,...
52439,59ca6ff6-d547-4195-967b-3d6b1db8a38c,NaN,ALTAN-LEJLIGHED - som mezzanin-lejlighed (i 1....,0a3f50a1-a398-32b8-e044-0003ba298018
52440,5f110503-f423-4510-9977-3ec14788495a,NaN,Attraktivt beliggende i det populære 'Jydekvar...,0a3f50a1-a398-32b8-e044-0003ba298018
52441,59ca6ff6-d547-4195-967b-3d6b1db8a38c,NaN,ALTAN-LEJLIGHED - som mezzanin-lejlighed (i 1....,0a3f50a1-a398-32b8-e044-0003ba298018
52442,5f110503-f423-4510-9977-3ec14788495a,NaN,Attraktivt beliggende i det populære 'Jydekvar...,0a3f50a1-a398-32b8-e044-0003ba298018


## We insert Water_view as an column. 

In [54]:
a = ['havudsigt', 'havkig', 'vandkig', 'udsigt over øresund', 'udsigt over vand', 'udsigt til vand', 'udkig til vand', 'vandudsigt']
pattern = '|'.join(a)
pattern

'havudsigt|havkig|vandkig|udsigt over øresund|udsigt over vand|udsigt til vand|udkig til vand|vandudsigt'

In [55]:
water_view_in_title = list(Beskrivelser.loc[Beskrivelser['title'].str.lower().str.contains(pattern) == True].index)


In [56]:
water_view_in_description = list(Beskrivelser.loc[Beskrivelser['description'].str.lower().str.contains(pattern) == True].index)


In [57]:
Beskrivelser['Water_view'] = 0 #We create our new column, and fill it with zeros. 

In [58]:
Beskrivelser.loc[water_view_in_title ,'Water_view'] = 1

In [59]:
Beskrivelser.loc[water_view_in_description ,'Water_view'] = 1

In [60]:
Beskrivelser.loc[Beskrivelser['Water_view'] == 1] #And we can now see that we found 1092 case-id with a Water_view. 

,case_id,title,description,address_id,Water_view
189,1986d031-5bcc-4486-ab74-8d9cf457407d,NaN,OMRÅDET:<BR />Velkommen til Amagers absolut be...,0a3f509d-2506-32b8-e044-0003ba298018,1
326,43ba3a32-fe53-4a1c-8b6a-f11b428ca450,LIEBHAVERI MED HAVUDSIGT OG PRIVAT P-PLADS,Velkommen til Amerika Have - et lejlighedskomp...,3e064cf7-dc70-4a3c-854b-09609db2c47f,1
441,bf27afe0-332e-48af-8b6c-4bc69103ddd5,Penthouse lejlighed med stor altan og vandudsigt,1 til 1-fremvisning med tilmeldingskrav 18/8 -...,8330ebb5-21cb-4ac7-9166-1434774db2b2,1
449,7ff8548a-2cf5-4d06-8ed4-a4174f522712,Penthouselejlighed med kæmpe altan vandudsigt ...,Her får du en ny bolig af den højeste standard...,9bc20a3f-67f4-495f-a325-d521f97af6ca,1
1036,7d92020c-ce15-4e8f-a30e-e66a60eeeb8f,NaN,LEJLIGHED: Hermed udbydes lejlighed til den kr...,0a3f50a1-3b52-32b8-e044-0003ba298018,1
...,...,...,...,...,...
52184,5290d366-4e23-4dfb-a9fd-7e07d05dcb8b,Flot udsigtslejlighed med altan på Sluseholmen,Flot udsigtslejlighed med altan på Sluseholmen...,1b974be1-7727-60d6-e044-0003ba298018,1
52192,1db71533-cf97-428b-8664-c19cb282a0f2,"Udsigt over vandet, grønne områder og egen p-p...",Lejligheden: Virkelig spændende og velindrette...,74325fe0-b1bd-4127-90b5-93dc740b2bcc,1
52307,468c1a29-c2ee-40af-acf0-7f1219f8dc43,NaN,"Med beliggenhed tæt på Amager Strandpark, metr...",4a78ea4c-0508-6ee8-e044-0003ba298018,1
52316,395c67e6-c152-4532-b0af-e700eee7e4aa,NaN,"Med beliggenhed tæt på Amager Strandpark, metr...",4a78ea4c-0509-6ee8-e044-0003ba298018,1


In [61]:
def search(df_in, df_words):
    
    df_input = df_in
    #First we lower the description text, so it is easy for us to search,
    df_input['description_low'] = df_input['description'].str.lower()

    for i in tqdm(range(0, len(df_words))):
        #Now we find in the description, when 'altan'(balcony) is mentioned
        df_input[df_words.loc[i, 'Column_name']] = df_input['description_low'].str.contains(df_words.loc[i, 'Search_string'], regex=False)
        
        #We make both new columns to 1 and 0, so 1 for true and 0 for False and NaN. 
        df_input[df_words.loc[i, 'Column_name']] = df_input[df_words.loc[i, 'Column_name']].fillna(0)
        df_input[df_words.loc[i, 'Column_name']] = df_input[df_words.loc[i, 'Column_name']].astype(int)


    
    #We now remove 1 from Balcony, if it also has 1 at Opp_for_balcony. Because then 'altan'(balcony) was mentioned in 'mulighed for altan'(opportunity for balcony).
    for i in tqdm(range(0, len(df_input))):
        if df_input.loc[i,'Opp_for_balcony'] == 1:
            df_input.loc[i, 'Balcony'] = 0

    #We now remove 1 from Terrace, if it also has 1 at Rooftop Terrace or Opp_for_Rooftop_Terrace or Projekt_std_text(This string has at least 1 balcony).
    for i in tqdm(range(0, len(df_input))):
        if df_input.loc[i,'Rooftop Terrace'] == 1 or df_input.loc[i,'Opp_for_Rooftop_Terrace'] == 1 or df_input.loc[i,'Projekt_std_text'] == 1:
            df_input.loc[i, 'Terrace'] = 0     
        

    #We now remove 1 from House Animals, if it also has 1 at No_House Animals.
    for i in tqdm(range(0, len(df_input))):
        if df_input.loc[i,'No_House_Animals1'] == 1 or df_input.loc[i,'No_House_Animals2'] == 1 or df_input.loc[i,'No_House_Animals3'] == 1 or df_input.loc[i,'No_House_Animals4'] == 1 or df_input.loc[i,'No_House_Animals5'] == 1:
            df_input.loc[i, 'House Animals'] = 0     

    
    df_input = df_input.drop(columns= ['title', 'description', 'description_low', 'No_House_Animals1', 'No_House_Animals2', 'No_House_Animals3', 'No_House_Animals4', 'No_House_Animals5', 'Projekt_std_text'])
    
    

    return df_input




In [62]:
words = pd.DataFrame(data=[['Balcony', 'altan'], ['Opp_for_balcony', 'mulighed for altan'], ['Rooftop Terrace', 'tagterasse'],['Opp_for_Rooftop_Terrace', str.lower('SØGES OM TILLADELSE TIL AT LAVE TAGTERRASSE')],['Terrace', 'terrasse'], ['Projekt_std_text', 'og flere har endda både altan og privat tagterrasse'], ['House Animals', 'husdyr'], ['No_House_Animals1', 'husdyr ej'], ['No_House_Animals2', 'husdyr ikke'], ['No_House_Animals3', 'ikke husdyr'], ['No_House_Animals4', 'husdyrhold er ikke tilladt'], ['No_House_Animals5', 'husdyr er ikke tilladt']], columns=['Column_name', 'Search_string'])
words

,Column_name,Search_string
0,Balcony,altan
1,Opp_for_balcony,mulighed for altan
2,Rooftop Terrace,tagterasse
3,Opp_for_Rooftop_Terrace,søges om tilladelse til at lave tagterrasse
4,Terrace,terrasse
5,Projekt_std_text,og flere har endda både altan og privat tagter...
6,House Animals,husdyr
7,No_House_Animals1,husdyr ej
8,No_House_Animals2,husdyr ikke
9,No_House_Animals3,ikke husdyr


In [63]:
Search_Beskrivelser = search(Beskrivelser, words)
Search_Beskrivelser

100%|██████████| 52444/52444 [00:02<00:00, 25118.16it/s]


,case_id,address_id,Water_view,Balcony,Opp_for_balcony,Rooftop Terrace,Opp_for_Rooftop_Terrace,Terrace,House Animals
0,da3c0f7d-e863-4e30-95f7-201a96065ec7,0a3f509d-26a5-32b8-e044-0003ba298018,0,0,0,0,0,0,0
1,9c5e7ed9-95f9-41c0-900f-b5e0d320e04d,0a3f509d-2680-32b8-e044-0003ba298018,0,0,0,0,0,0,0
2,d7e43f7e-edb6-46a6-a766-de1ddfa0e284,0a3f50a2-a0c3-32b8-e044-0003ba298018,0,1,0,0,0,0,0
3,90be8918-b3ba-40d1-8187-15f997c263e0,0a3f50a2-a15d-32b8-e044-0003ba298018,0,0,0,0,0,0,0
4,c2faf077-406a-4f1b-acd0-719c9e9946ab,0a3f50a2-a160-32b8-e044-0003ba298018,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
52439,59ca6ff6-d547-4195-967b-3d6b1db8a38c,0a3f50a1-a398-32b8-e044-0003ba298018,0,1,0,0,0,0,0
52440,5f110503-f423-4510-9977-3ec14788495a,0a3f50a1-a398-32b8-e044-0003ba298018,0,1,0,0,0,0,0
52441,59ca6ff6-d547-4195-967b-3d6b1db8a38c,0a3f50a1-a398-32b8-e044-0003ba298018,0,1,0,0,0,0,0
52442,5f110503-f423-4510-9977-3ec14788495a,0a3f50a1-a398-32b8-e044-0003ba298018,0,1,0,0,0,0,0


In [64]:
Search_Beskrivelser.sum()

case_id                    da3c0f7d-e863-4e30-95f7-201a96065ec79c5e7ed9-9...
address_id                 0a3f509d-26a5-32b8-e044-0003ba2980180a3f509d-2...
Water_view                                                              1092
Balcony                                                                25815
Opp_for_balcony                                                          101
Rooftop Terrace                                                           35
Opp_for_Rooftop_Terrace                                                    1
Terrace                                                                10891
House Animals                                                           1398
dtype: object

In [65]:
raw_data_final = pd.merge(raw_data2, Search_Beskrivelser.drop_duplicates(), how = 'inner', on = ['case_id', 'address_id'] )


We adjust our House Animal variable, becuase we do not think that it would be mentioned in the Sale description for a Villa and terraced house, full year plot. 

In [66]:
animal_list = list(raw_data_final.loc[raw_data_final['House Animals']==0].loc[~raw_data_final['type'].isin(['condo', 'villa apartment'])].index)

In [67]:
raw_data_final.loc[animal_list,['House Animals']] = 1

In [68]:
raw_data_final

,case_id,type,energy_law_applies,energy_label_classification,presale,num_floors,num_rooms,living_area,basement_area,lot_area,year_built,sold_price,registered_price,address_id,rental,latest_status,weighted_area,closed_or_sold_at,original_price,unit_id,usage_code,living_area.1,business_area,num_rooms.1,updated_at,address_id.1,num_floors.1,year_built.1,year_renovated,ownership,building_id,num_toilets,toilet_condition_code,num_bathrooms,bathroom_condition_code,kitchen_condition_code,floor_id,status,property_number,building_id.1,building_number,entry_address_id,usage_code.1,year_built.2,year_renovated.1,outer_wall_code,roof_code,additional_outer_wall_mat,additional_roof_mat,asbestos_materials,total_area,living_area_total,business_area_total,other_area,heating_code,kitchen_amount,floors,updated_at.1,additional_heating_code,ownership.1,total_built_area,built_in_garage_area,built_in_carport_area,built_in_outhouse_area,conservatory_area,property_number.1,status.1,weighted_area.1,unit_usage_code_beskrivelse,address_id.2,is_apartment,unit_living_area,full_floor_living_area,roof_living_area,roof_other,basement_living_area,basement_other,garage_outhouse_built_in,garage_outhouse_free,carport,conservatory,annex,weighted_area.2,toilet_beskrivelse,køkken_beskrivelse,tag_beskrivelse,ydervægsmaterialebeskrivelse,heating_code.1,heating_code_additional,badeværelsebeskrivelse,postnradgadr,postnummernavn,kommunekode,kommune,vejnavn,husnr,dør,etage,gstkvhx_kommunekode_vejkode_husnr_x,ID_Count,school,roadtrain,junction,daycare,metro,doctor,soccerfield,hospital,stop,lake,supermarket,pharmacy,strain,airport,train,library,publicbath,coast,sportshall,forest,Location,dingeo_link,Radonrisiko,Støjmåling,Oversvømmelsesrisiko_skybrud,Meter_over_havet,Indbrudsrisiko,Bevaringsværdi,Største_parti,Valgdeltagelse,Afstemningsområde,AVM_pris,latest_prop_valuation,Year_Week,Year_Months,Year_Quarter,Kort rente,Lang rente,OMXC 25,Consumer price index,ForcedSalesOfPropertyInRegionHovedstaden,AnnualisedAgreedRate,ElPrice_incl_moms_4000kWh,BenzinPrice_incl_moms,DieselPrice_incl_moms,unemployedRate,Population,Water_view,Balcony,Opp_for_balcony,Rooftop Terrace,Opp_for_Rooftop_Terrace,Terrace,House Animals
0,19053496-b401-48cd-a1cd-4dbad107dee6,condo,0.0,NaN,False,NaN,2.0,76.0,NaN,NaN,1700.0,6450000,NaN,bbe10eeb-cddc-44d3-aebb-b0b11e31862b,False,sold,76.000000,2019-08-01 12:53:49.137000,NaN,3ca52175-6dbc-4c0a-ba66-0e4075b67add,140.0,76.0,NaN,2.0,2020-09-26 22:31:08.520000,bbe10eeb-cddc-44d3-aebb-b0b11e31862b,3.0,1700.0,NaN,private,156a3021-31be-4a7f-b9ee-2b2e1bd016e5,2.0,T,1.0,V,E,a7b96403-d996-44c3-b8b4-888f5297c799,primary,536572.0,156a3021-31be-4a7f-b9ee-2b2e1bd016e5,1,0a3f507b-01bf-32b8-e044-0003ba298018,140,1700.0,NaN,1,5,NaN,NaN,NaN,3558,3808.0,99.0,NaN,1,39.0,3.0,2019-01-25 07:19:26.310000,90.0,other,1186.0,NaN,NaN,NaN,NaN,536572.0,primary,76.0,"Etagebolig-bygning, flerfamiliehus eller tofam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vandskyllende toilet i enheden,Eget køkken med afløb,Tegl,Mursten,Fjernvarme/blokvarme,(UDFASES) Bygningen har ingen supplerende varme,Badeværelse i enheden,1401,København K,101,København,Strandgade,44B,tv,2.0,01016992_44D__2__tv,0,435.13,5739.46,5707.89,542.49,490.44,459.07,1975.85,3506.60,331.89,670.57,384.65,480.76,1957.41,7286.50,1957.41,481.06,2252.54,100.07,979.67,6590.06,Strandgade44BKøbenhavn,https://www.dingeo.dk/adresse/1401-København-K...,Meget lav,55-60 dB,kan være risiko,2.0 meter,lav,0.0,enhedslisten,87.3,3. Syd,5.224.446,1900000.0,2019W31,2019M08,2019Q3,-0.66980,1.12240,1124,103.5,22,1.606,221.548060,12.712222,11.012222,4.3,626350.0,0,0,0,0,0,0,0
1,5a7e3ae4-cd86-4c75-a7d3-dca07bd7f0f8,condo,0.0,c,False,NaN,2.0,50.0,NaN,NaN,1897.0,3165000,3165000.0,0a3f509e-0bb0-32b8-e044-0003ba298018,False,registered,50.000000,2022-03-20 17:47:48.000000,NaN,54e9b180-c28f-432d-bf72-1112ad2c2fe8,140.0,50.0,NaN,2.0,2018-12-29 21:09:02.493000,0a3f509e-0bb0-32b8-e044-0003ba298018,6.0,1897.0,NaN,private,568dd859-0675-48d0-bd30-3af

In [69]:
#We save the final raw data

#raw_data_final.to_excel("raw_data_final.xlsx")

## We add the Square meter price, and save our raw data file

In [87]:
raw_data_final = read_saved_excel("raw_data_final.xlsx")

In [71]:
# def Get_Prices(df_input):
#     #We make an empty column for the m2 prices
#     df_input['m2Price'] = np.nan
#     #And we make a for-loop to insert the m2 prices. But we search first for if there are registered a 'registered_price', because then we want to use this price.
#     #Because the sold_price is the selling price reported by the broker and registered_price is the registered price.
#     #Most often they are the same, but there can be deviations sometimes, the registered price is probably the most reliable
#     for i in tqdm(range(0, len(df_input))):

#         if df_input['living_area'].isna()[i] == False:
#             if df_input['registered_price'].isna()[i] == False:
#                 df_input.loc[i, 'm2Price'] = df_input.loc[i, 'registered_price'] / df_input.loc[i, 'living_area']
#             else:
#                 df_input.loc[i, 'm2Price'] = df_input.loc[i, 'sold_price'] / df_input.loc[i, 'living_area']

#         else:
#             if df_input['living_area.1'].isna()[i] == False:
#                 if df_input['registered_price'].isna()[i] == False:
#                     df_input.loc[i, 'm2Price'] = df_input.loc[i, 'registered_price'] / df_input.loc[i, 'living_area.1']
#                 else:
#                     df_input.loc[i, 'm2Price'] = df_input.loc[i, 'sold_price'] / df_input.loc[i, 'living_area.1']
#             else:
#                 df_input.loc[i, 'm2Price'] = np.nan

#     return df_input

# We add the coordinates to the dataset 

In [89]:
raw_data_final_final = pd.merge(raw_data_final, coordinates.drop_duplicates('address_id'), how='left', on = 'address_id')


In [90]:
raw_data_final_final.to_excel('raw_data_final_final.xlsx')

* ## Check balcony if sold more than 1 time in the period
We check if some of the duplicates, have been sold without and with an balcony, then it should have come the latter. 

In [74]:
def check_altan(df_input):
    dup2 = df_input.groupby(df_input.columns.tolist()[1], as_index=False, dropna=False).size() #We look for duplicates in column: 'address_id' and add a new column: size, which we can tell how many time it is mentioned.
    a = dup2[dup2['size']>1] #And we can now look for the columns who is duplicated 
    a = a.reset_index()
    a = a.drop(columns=['index'])
    a['sum'] = np.nan

    for i in tqdm(range(0,len(a))):
        
        b = df_input[df_input['address_id'].str.contains(a.loc[i, 'address_id'])]
        a.loc[i, 'sum'] = b['Balcony'].sum()
        if a.loc[i, 'sum'] == 0.0:
            a = a.drop([i])
        elif a.loc[i, 'sum'] == a.loc[i, 'size']:
            a = a.drop([i])

    return a

In [75]:
#check = check_altan(Search_Beskrivelser)

In [76]:
check

NameError: name 'check' is not defined

In [ ]:
check.to_excel("ListeMedAltanerDerErBygget.xlsx")

In [ ]:
Search_Beskrivelser[Search_Beskrivelser['address_id']=='029b1abb-ab74-4519-8669-7df3b83f0771']

,case_id,address_id,Water_view,Balcony,Opp_for_balcony,Rooftop Terrace,Opp_for_Rooftop_Terrace,Terrace,House Animals
50832,2362eaaa-8696-49ca-beae-c8f52996cd17,029b1abb-ab74-4519-8669-7df3b83f0771,0,0,0,0,0,0,0
50833,fc5b8e3f-6a60-47bd-a3a1-af8f13bb62e7,029b1abb-ab74-4519-8669-7df3b83f0771,0,1,0,0,0,0,0


In [ ]:
raw_data2[raw_data2['address_id']=='029b1abb-ab74-4519-8669-7df3b83f0771']

,case_id,type,energy_law_applies,energy_label_classification,presale,num_floors,num_rooms,living_area,basement_area,lot_area,year_built,sold_price,registered_price,address_id,rental,latest_status,weighted_area,closed_or_sold_at,original_price,unit_id,usage_code,living_area.1,business_area,num_rooms.1,updated_at,address_id.1,num_floors.1,year_built.1,year_renovated,ownership,building_id,num_toilets,toilet_condition_code,num_bathrooms,bathroom_condition_code,kitchen_condition_code,floor_id,status,property_number,building_id.1,building_number,entry_address_id,usage_code.1,year_built.2,year_renovated.1,outer_wall_code,roof_code,additional_outer_wall_mat,additional_roof_mat,asbestos_materials,total_area,living_area_total,business_area_total,other_area,heating_code,kitchen_amount,floors,updated_at.1,additional_heating_code,ownership.1,total_built_area,built_in_garage_area,built_in_carport_area,built_in_outhouse_area,conservatory_area,property_number.1,status.1,weighted_area.1,unit_usage_code_beskrivelse,address_id.2,is_apartment,unit_living_area,full_floor_living_area,roof_living_area,roof_other,basement_living_area,basement_other,garage_outhouse_built_in,garage_outhouse_free,carport,conservatory,annex,weighted_area.2,toilet_beskrivelse,køkken_beskrivelse,tag_beskrivelse,ydervægsmaterialebeskrivelse,heating_code.1,heating_code_additional,badeværelsebeskrivelse,postnradgadr,postnummernavn,kommunekode,kommune,vejnavn,husnr,dør,etage,gstkvhx_kommunekode_vejkode_husnr_x,ID_Count,school,roadtrain,junction,daycare,metro,doctor,soccerfield,hospital,stop,lake,supermarket,pharmacy,strain,airport,train,library,publicbath,coast,sportshall,forest,Location,dingeo_link,Radonrisiko,Støjmåling,Oversvømmelsesrisiko_skybrud,Meter_over_havet,Indbrudsrisiko,Bevaringsværdi,Største_parti,Valgdeltagelse,Afstemningsområde,AVM_pris,Year_Week,Year_Months,Year_Quarter,Kort rente,Lang rente,OMXC 25,Consumer price index,ForcedSalesOfPropertyInRegionHovedstaden,AnnualisedAgreedRate,ElPrice_incl_moms_4000kWh,BenzinPrice_incl_moms,DieselPrice_incl_moms,unemployedRate,Population
7923,2362eaaa-8696-49ca-beae-c8f52996cd17,condo,0.0,NaN,True,1.0,4.0,109.0,NaN,NaN,2017.0,4745000,NaN,029b1abb-ab74-4519-8669-7df3b83f0771,False,sold,NaN,2017-03-02 07:25:14.603000,NaN,d4fbe401-15c1-4f97-8797-032c0c81fc7f,140.0,109.0,NaN,2.0,2019-01-25 20:10:48.160000,029b1abb-ab74-4519-8669-7df3b83f0771,8.0,2017.0,NaN,private,70bd32eb-9dfe-4043-9b7e-02ce83178d53,1.0,T,1.0,V,E,886b3241-aa05-4cad-8a64-5edddaeb24ed,primary,140334.0,70bd32eb-9dfe-4043-9b7e-02ce83178d53,8,eee0d5b8-720c-4590-ad19-90b51ef4ab34,140,2017.0,NaN,1,1,NaN,NaN,NaN,21863,22814.0,144.0,233.0,1,192.0,8.0,2020-07-20 06:58:54.758000,NaN,trust,2765.0,NaN,NaN,NaN,NaN,140334.0,construction,NaN,"Etagebolig-bygning, flerfamiliehus eller tofam...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vandskyllende toilet i enheden,Eget køkken med afløb,Tagpap med lille hældning,Mursten,Fjernvarme/blokvarme,NaN,Badeværelse i enheden,2450,København SV,101,København,Martin Luther Kings Vej,1,tv,2.0,01018843___5__2__tv,7923,1537.47,4775.0,3462.98,337.66,1884.12,1190.9,1827.56,4372.74,782.24,1980.24,518.17,1326.6,1652.75,8031.34,2778.8,1825.39,2313.69,98.34,2543.22,4987.9,Martin Luther Kings Vej1København,https://www.dingeo.dk/adresse/2450-København-S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017W09,2017M03,2017Q1,-0.44665,2.46498,1060,101.1,29,2.126,239.420729,11.806667,10.066,5.0,601448.0
7924,fc5b8e3f-6a60-47bd-a3a1-af8f13bb62e7,condo,0.0,a2015,True,1.0,3.0,109.0,NaN,NaN,2017.0,4745000,4745000.0,029b1abb-ab74-4519-8669-7df3b83f0771,False,registered,NaN,2017-01-19 10:10:04.600000,NaN,d4fbe401-15c1-4f97-8797-032c0c81fc7f,140.0,109.0,NaN,2.0,2019-01-25 20:10:48.160000,029b1abb-ab74-4519-8669-7df3b83f0771,8.0,2017.0,NaN,private,70bd32eb-9dfe-4043-9b7e-02ce83178d53,1.0,T,1.0,V,E,886b3241-aa05-4cad-8a64-5edddaeb24ed,primary,140334.0,70bd32eb-9dfe-4043-9b7e-02ce83178d53,8,eee0d5b8-720c-4590-ad19-90b51ef4ab34,140,2017.0,NaN,1,1,NaN,NaN,

In [ ]:


dup2 = Search_Beskrivelser.groupby(Search_Beskrivelser.columns.tolist()[1], as_index=False, dropna=False).size() #We look for duplicates in column: 'address_id' and add a new column: size, which we can tell how many time it is mentioned.
a = dup2[dup2['size']>1] #And we can now look for the columns who is duplicated 
a

,address_id,size
3,00201f18-28dc-4305-a4e2-b6889a59816c,2
5,00250fd1-260d-4c6b-b44f-5aa0fe209e70,2
7,002859e3-2a2e-4ca3-a4b0-cd7e36d428e0,2
10,009a6932-7381-4504-be6a-3989c19039f7,3
12,00b98342-b1b3-41c1-9386-ee2b015ba33b,2
...,...,...
42516,ff27ceae-43c8-4c0b-a1a2-3c2a369cf77b,2
42527,ff7bf989-a121-415a-b456-77763dd481bf,2
42534,ffafb1d1-0be8-4f43-a29d-f1218af962be,2
42544,fff4c171-fed2-489d-a34b-5c4117e50564,2


In [ ]:
dup2 = raw_data2.groupby(raw_data2.columns.tolist()[13], as_index=False, dropna=False).size() #We look for duplicates in column: 'address_id' and add a new column: size, which we can tell how many time it is mentioned.
a = dup2[dup2['size']>1] #And we can now look for the columns who is duplicated 
a

,address_id,size
3,00201f18-28dc-4305-a4e2-b6889a59816c,2
5,00250fd1-260d-4c6b-b44f-5aa0fe209e70,2
7,002859e3-2a2e-4ca3-a4b0-cd7e36d428e0,2
10,009a6932-7381-4504-be6a-3989c19039f7,3
12,00b98342-b1b3-41c1-9386-ee2b015ba33b,2
...,...,...
42516,ff27ceae-43c8-4c0b-a1a2-3c2a369cf77b,2
42527,ff7bf989-a121-415a-b456-77763dd481bf,2
42534,ffafb1d1-0be8-4f43-a29d-f1218af962be,2
42544,fff4c171-fed2-489d-a34b-5c4117e50564,2
